In [1]:
from preprocess import preprocess_pipeline

data = preprocess_pipeline()

2025-08-21 17:40:30.339023: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755787230.355942   94071 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755787230.360555   94071 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1755787230.372341   94071 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755787230.372357   94071 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755787230.372359   94071 computation_placer.cc:177] computation placer alr

1. Include only first admissions: N=32513
2. Exclusion by ages: N=25548
3. Include only patients who admitted for at least 24 hours: N=25168
4. Exclude patients who died within 48-hours of admission: N=24825


/home/bnet/ronsheinin/MLHC/MLHC/preprocess.py:203: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  pivot = pd.pivot_table(merged, index=['subject_id', 'hadm_id', pd.Grouper(key='charttime', freq=PREDICT_FREQ)],
/home/bnet/ronsheinin/MLHC/MLHC/preprocess.py:263: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train.drop(columns=["mort_30day","prolonged_stay","readmission_30day"], axis=1, inplace=True)
/home/bnet/ronsheinin/MLHC/MLHC/preprocess.py:267: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_val.drop(columns=["mort_30day","prolonged_stay","readmission_30day"], a

Pooled embeddings shape: 24798
Number of subjects: 24798


In [2]:
from Dataset import PatientDataset
import torch
from torch.utils.data import DataLoader
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
k=10

train_labels = torch.tensor(data["y_train"][['mort_30day', 'prolonged_stay', 'readmission_30day']].values, dtype=torch.float32).to(DEVICE)
val_labels = torch.tensor(data["y_val"][['mort_30day', 'prolonged_stay', 'readmission_30day']].values, dtype=torch.float32).to(DEVICE)
test_labels = torch.tensor(data["y_test"][['mort_30day', 'prolonged_stay', 'readmission_30day']].values, dtype=torch.float32).to(DEVICE)


batch_size = 256
datasets = {x: PatientDataset(d, y, core=data["padded_tensor_core"], padding_mask=m, padding_mask_core=data["padding_mask_core"], k=k ,notes=n) for x, d, y, m, n in
        zip(['train', 'val', 'test'], [data["padded_tensor_train"], data["padded_tensor_val"], data["padded_tensor_test"]],
            [train_labels, val_labels, test_labels],
            [data["padding_mask_train"], data["padding_mask_val"], data["padding_mask_test"]],
            [torch.stack(data["notes_df_train"].embeddings.values.tolist()),
             torch.stack(data["notes_df_val"].embeddings.values.tolist()),
             torch.stack(data["notes_df_test"].embeddings.values.tolist())])}
dataloaders = {x: DataLoader(datasets[x], batch_size=batch_size, shuffle=True) for x in ['train', 'val', 'test']}

In [4]:
from Model import GraphGRUMortalityModel
model = GraphGRUMortalityModel(input_dim=data["padded_tensor_train"].shape[2], hidden_dim=64, n1_gat_layers=1, n2_gru_layers=2, X_core=data["padded_tensor_core"], 
                               core_padding_mask=data["padding_mask_core"], num_heads=4, dropout=0.2, seq_len=data["padded_tensor_train"].shape[1], k=k, gnn_flag=True).to(DEVICE)

model.train_all(dataloaders, datasets, epochs=10, learning_rate=1e-3)
print("Training completed. Validating on test set...")

acc, auc, pr =  model.validate(dataloaders['test'], datasets['test'])
print(f'Test Accuracy in model: {acc:.4f}, AUC: {auc:.4f}, AP: {pr:.4f}')

Pos weight: 6.8738
Starting epoch 1/10


  0%|          | 0/77 [00:00<?, ?it/s]

100%|██████████| 77/77 [00:51<00:00,  1.49it/s]


Epoch 1/10, Loss: 1.0506


100%|██████████| 10/10 [00:06<00:00,  1.53it/s]


...

Epoch 10/10, Loss: 0.6633


100%|██████████| 10/10 [00:06<00:00,  1.52it/s]


Validation Accuracy: 0.8427 | AUC: 0.8353 | AP: 0.4888
Training completed. Validating on test set...


100%|██████████| 10/10 [00:06<00:00,  1.53it/s]

Test Accuracy in model: 0.8601, AUC: 0.8725, AP: 0.5342


In [2]:
import torch
import torch.nn as nn   
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
notes_tensor_train = torch.stack(data["notes_df_train"]["embeddings"].values.tolist(), dim=0).to(DEVICE)
notes_tensor_test = torch.stack(data["notes_df_test"]["embeddings"].values.tolist(), dim=0).to(DEVICE)

labels_train = torch.tensor(data["y_train"]["mort_30day"].values, dtype=torch.float32).to(DEVICE)
labels_test = torch.tensor(data["y_test"]["mort_30day"].values, dtype=torch.float32).to(DEVICE)


In [3]:
from sklearn.metrics import roc_auc_score, average_precision_score
from torch.utils.data import DataLoader, TensorDataset
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim=512, dropout=0.3):
        super(MLP, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim // 2, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        return self.layers(x).squeeze()

# Initialize model
input_dim = notes_tensor_train.shape[1]
model = MLP(input_dim).to(DEVICE)

# Training setup
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
model.train()
epochs = 100

# Create dataset and dataloader for training
train_dataset = TensorDataset(notes_tensor_train[:labels_train.shape[0]], labels_train)
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
for epoch in range(epochs):
    for notes, labels in train_loader:
        notes, labels = notes.to(DEVICE), labels.to(DEVICE)
        
        optimizer.zero_grad()
        outputs = model(notes)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    
    print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluation
model.eval()
with torch.no_grad():
    test_outputs = model(notes_tensor_test)
    test_probs = test_outputs.cpu().numpy()
    test_labels = labels_test.cpu().numpy()
    
    roc_auc = roc_auc_score(test_labels, test_probs)
    ap_score = average_precision_score(test_labels, test_probs)
    
    print(f'Test ROC-AUC: {roc_auc:.4f}')
    print(f'Test AP: {ap_score:.4f}')

Epoch [1/100], Loss: 0.2686
Epoch [2/100], Loss: 0.4134
Epoch [3/100], Loss: 0.4066
Epoch [4/100], Loss: 0.2905
Epoch [5/100], Loss: 0.2815
Epoch [6/100], Loss: 0.3076
Epoch [7/100], Loss: 0.2741
Epoch [8/100], Loss: 0.1840
Epoch [9/100], Loss: 0.1674
Epoch [10/100], Loss: 0.1917
Epoch [11/100], Loss: 0.2063
Epoch [12/100], Loss: 0.4376
Epoch [13/100], Loss: 0.2411
Epoch [14/100], Loss: 0.2334
Epoch [15/100], Loss: 0.2155
Epoch [16/100], Loss: 0.1334
Epoch [17/100], Loss: 0.2676
Epoch [18/100], Loss: 0.4278
Epoch [19/100], Loss: 0.2282
Epoch [20/100], Loss: 0.3209
Epoch [21/100], Loss: 0.3917
Epoch [22/100], Loss: 0.3429
Epoch [23/100], Loss: 0.2995
Epoch [24/100], Loss: 0.3291
Epoch [25/100], Loss: 0.5416
Epoch [26/100], Loss: 0.2831
Epoch [27/100], Loss: 0.2806
Epoch [28/100], Loss: 0.2724
Epoch [29/100], Loss: 0.2515
Epoch [30/100], Loss: 0.2461
Epoch [31/100], Loss: 0.1546
Epoch [32/100], Loss: 0.2422
Epoch [33/100], Loss: 0.1155
Epoch [34/100], Loss: 0.4550
Epoch [35/100], Loss: 0

In [13]:
data["y_train"]

,subject_id,mort_30day,prolonged_stay,readmission_30day
0,3,0,1,0
1,4,0,0,0
2,9,1,0,0
3,11,0,1,0
4,18,0,0,0
...,...,...,...,...
19833,99983,0,0,0
19834,99985,0,1,0
19835,99991,0,1,0
19836,99992,0,0,0
